# Metadata

**L1 Taxonomy** - Background Processes

**L2 Taxonomy** - Job Scheduling

**Subtopic** - Implementing a timeout for jobs that overrun and scheduling follow-up actions

**Use Case** - Develop a Python script that schedules a list of jobs with a predetermined timeout. If a job exceeds the timeout, the script should stop the job and schedule a follow-up action. The follow-up action should involve logging the job details to a local file for further analysis. All jobs and follow-up actions should be defined as Python functions within the script.

**Programming Language** - Python

**Target Model** - GPT-4o

# Setup

```requirements.txt
```


# Prompt

Build a Python script that schedules a list of jobs with a fixed per-job timeout. Each job is a Python function defined in the same script. When a job exceeds the timeout, stop it and immediately schedule a follow-up action that logs the job details to a local file for later analysis. For jobs that finish in time, record their result. Use only the standard library.

Input comes from in-script calls to your API; there is no stdin. Jobs and the follow-up action are regular Python callables. The scheduler must run jobs one at a time with a hard timeout per job.

Input format and constraints

* Jobs are provided as a list of pairs (job_id: str, func: Callable[[], Any]).
* Timeout is a positive float in seconds and applies to each job separately.
* The follow-up action is a callable that accepts the timed-out job’s details and appends a log line to a text file.
* All code runs on CPython, cross-platform (Windows, macOS, Linux).

Expected output format

* The scheduler returns a list of per-job records with fields:

  * job_id (str)
  * status in {"ok", "timeout", "error"}
  * duration_sec (float, rounded or raw)
  * result (if status == "ok") or error (repr of exception)

Examples

Jobs:

```
def slow_job():
    import time
    time.sleep(2)
    return "done"

def fast_job():
    return 42
```

Run:

```
records = schedule_jobs(
    jobs=[("J1", fast_job), ("J2", slow_job)],
    timeout=1.0,
    follow_up=default_follow_up_logger,
    log_file="job_followups.log",
)
```

Sample outcomes:

* For J1: status "ok", result 42
* For J2: status "timeout", follow-up logger appends a line to job_followups.log


# Requirements

### Explicit and implicit points

* Enforce a hard timeout per job. Overrunning jobs must be stopped and marked "timeout".
* Immediately invoke a follow-up action for timed-out jobs that appends a line to a local log file.
* Run jobs in isolation so they can be stopped safely on timeout (use multiprocessing, not threading; do not use OS-specific signals).
* Process jobs sequentially in input order for determinism.
* Do not print or log from inside the scheduler except via the provided follow-up callable.
* All jobs and the follow-up action are defined in the same script for testing.
* Results and errors must be captured and returned in a structured list.

### Solution expectations

* Use multiprocessing.Process to execute each job in its own child process and multiprocessing.Queue or Pipe to collect a result or exception.
* Use process.join(timeout) and process.terminate() to enforce the timeout. Ensure the child process is cleaned up (join after terminate).
* Measure per-job duration with time.monotonic().
* Implement a default follow-up that appends a single line per timeout with timestamp, job\_id, elapsed time, and a reason string. Use append mode and ensure parent directories exist.
* Open files with context managers. Use text mode UTF-8.
* Make behavior deterministic and cross-platform. Guard top-level runnable code with `if __name__ == "__main__":` when needed.

### Signatures of expected functions

```python
from collections.abc import Callable
from typing import Any

def run_job_with_timeout(func: Callable[[], Any], timeout: float) -> tuple[str, float, Any | BaseException]:
    """Run func in a child process with a hard timeout.
    Returns (status, duration_sec, payload) where:
      - status is "ok", "timeout", or "error"
      - payload is result (for "ok") or the exception instance (for "error")
    """

def default_follow_up_logger(job_id: str, reason: str, duration_sec: float, log_file: str) -> None:
    """Append a line describing the timed-out job to log_file."""

def schedule_jobs(
    jobs: list[tuple[str, Callable[[], Any]]],
    timeout: float,
    follow_up: Callable[[str, str, float, str], None],
    log_file: str,
) -> list[dict]:
    """Run jobs sequentially with timeout and follow-up on timeout.
    Returns a list of dicts with keys: job_id, status, duration_sec, result or error.
    """
```

### Edge case behavior

* timeout <= 0 -> raise ValueError("invalid timeout").
* Empty job list -> return empty list and create no logs.
* Job raises an exception -> status "error", include repr(exc) in error.
* Job returns a non-picklable result -> treat as "error" (child fails to send result); include a descriptive error repr.
* Follow-up logging path unwritable -> propagate the exception from the follow-up call.
* Very long-running job -> must be terminated on timeout and not leak child processes.
* Duplicate job_ids are allowed; records must preserve input order.

### Constraints

* Use only Python standard library: multiprocessing, time, os, io, pathlib, typing, collections.abc, traceback, json (optional for formatting), datetime.
* No external services, daemons, or system-specific signals.
* Time complexity is linear in number of jobs; memory use is bounded aside from job results.
* Cross-platform: use the default start method or explicitly choose "spawn" where necessary.

### Important notes

* Validate inputs:

  * jobs must be a list of (str, callable); raise ValueError("invalid jobs") otherwise.
  * timeout must be a float > 0.
  * follow_up must be callable; log_file must be a non-empty string.
* The scheduler must not mutate the provided job callables.
* The follow-up action must run only for timeouts, not for ordinary errors.
* Tests will assert deterministic ordering of records and that timed-out jobs are terminated and logged.
* Do not rely on wall-clock time for correctness; use time.monotonic() for durations.


In [ ]:
# code

from collections.abc import Callable
from datetime import datetime
from multiprocessing import Process, Queue
from pathlib import Path
from queue import Empty
from typing import Any, Tuple
import time


def _child_worker(func: Callable[[], Any], q: Queue) -> None:
    """
    Execute func and put a tuple ('ok', result) or ('error', exception_instance) into q.
    Handles the case where the result itself is not picklable by sending a synthetic error.
    """
    try:
        result = func()
        try:
            q.put(("ok", result))
        except Exception as put_error:
            # Result not picklable or queue serialization failed.
            q.put(("error", RuntimeError(
                f"result not picklable: {put_error!r}")))
    except Exception as e:
        # Function raised; send the exception instance (usually picklable)
        try:
            q.put(("error", e))
        except Exception as put_error:
            # As a last resort, send a simple, picklable exception
            q.put(("error", RuntimeError(
                f"child failed to send exception: {put_error!r}")))


# Public API
def run_job_with_timeout(func: Callable[[], Any], timeout: float) -> Tuple[str, float, Any | BaseException]:
    """Run func in a child process with a hard timeout.

    Returns (status, duration_sec, payload) where:
      - status is "ok", "timeout", or "error"
      - payload is result (for "ok") or the exception instance (for "error")

    Notes:
      - Uses multiprocessing with a top-level worker for cross-platform 'spawn'.
      - Uses time.monotonic() for duration measurement.
    """
    if not isinstance(timeout, float) or timeout <= 0.0:
        raise ValueError("invalid timeout")

    q: Queue = Queue(maxsize=1)
    p = Process(target=_child_worker, args=(func, q))
    start = time.monotonic()
    p.start()
    p.join(timeout)
    duration = time.monotonic() - start

    if p.is_alive():
        # Timeout: hard stop, ensure cleanup
        p.terminate()
        p.join()
        # Parent should finalize the queue properly even on timeout.
        q.close()
        q.join_thread()
        return "timeout", duration, None

    # Child exited; retrieve outcome non-blockingly with a small timeout
    try:
        status, payload = q.get(timeout=1.0)
    except Empty:
        status, payload = "error", RuntimeError(
            "no result received from child")
    finally:
        q.close()
        q.join_thread()

    return status, duration, payload


def default_follow_up_logger(job_id: str, reason: str, duration_sec: float, log_file: str) -> None:
    """Append a single line describing the timed-out job to log_file.

    The line includes a timestamp, job_id, elapsed time, and reason.
    Ensures parent directories exist when the log_file includes a directory.
    Uses UTF-8 text and context managers.
    """
    if not isinstance(log_file, str) or not log_file:
        raise ValueError("invalid log_file")

    p = Path(log_file)
    # Create parent directory if present (no-op when file is in CWD)
    if p.parent and str(p.parent) not in ("", "."):
        p.parent.mkdir(parents=True, exist_ok=True)

    ts = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    line = f"{ts} - Job ID: {job_id}, Reason: {reason}, Elapsed Time: {duration_sec:.6f} seconds\n"
    with p.open("a", encoding="utf-8") as f:
        f.write(line)


def schedule_jobs(
    jobs: list[tuple[str, Callable[[], Any]]],
    timeout: float,
    follow_up: Callable[[str, str, float, str], None],
    log_file: str,
) -> list[dict]:
    """Run jobs sequentially with a per-job hard timeout and follow-up on timeouts.

    Returns a list of dicts with keys: job_id, status, duration_sec, and one of:
      - result (if status == "ok")
      - error (repr of exception, if status == "error")

    Behavior:
      - Validates inputs as per spec.
      - Enforces hard timeout with process.terminate() and cleanup join.
      - Uses time.monotonic() for durations.
      - Calls follow_up only for timeouts; exceptions propagate from follow_up.
      - Maintains deterministic ordering of records.
    """
    # Validate inputs
    if not isinstance(jobs, list) or not all(
        isinstance(j, tuple) and len(j) == 2 and isinstance(
            j[0], str) and callable(j[1])
        for j in jobs
    ):
        raise ValueError("invalid jobs")

    if not isinstance(timeout, float) or timeout <= 0.0:
        raise ValueError("invalid timeout")

    if not callable(follow_up) or not isinstance(log_file, str) or not log_file:
        raise ValueError("invalid follow_up or log_file")

    records: list[dict] = []
    if not jobs:
        # Empty job list: return empty records and do not create logs
        return records

    for job_id, func in jobs:
        status, duration, payload = run_job_with_timeout(func, timeout)
        rec: dict = {"job_id": job_id,
                     "status": status, "duration_sec": duration}

        if status == "ok":
            rec["result"] = payload
        elif status == "error":
            rec["error"] = repr(payload)
        elif status == "timeout":
            # Invoke follow-up immediately; exceptions propagate by design
            follow_up(job_id, "timeout", duration, log_file)
        else:
            # Should never occur; defensive
            rec["error"] = repr(RuntimeError(f"unknown status: {status}"))

        records.append(rec)

    return records

In [ ]:
# tests

"""
Unit tests for job scheduling with timeout, logging, and error handling.
"""

import unittest
import os
import time
import tempfile
from typing import Callable
from pathlib import Path
from main import (
    schedule_jobs,
    default_follow_up_logger,
)


def fast_job() -> int:
    """Returns quickly with a valid result."""
    return 42


def slow_job() -> str:
    """Sleeps for 2 seconds and returns a string."""
    time.sleep(2)
    return "done"


def error_job() -> None:
    """Raises a RuntimeError immediately."""
    raise RuntimeError("job failed")


def non_picklable_job() -> Callable:
    """Returns a lambda (non-picklable)."""
    return lambda x: x


class TestJobScheduler(unittest.TestCase):
    """Test cases for the job scheduler module."""

    def setUp(self) -> None:
        """Create a temporary file for logging follow-ups."""
        self.temp_dir = tempfile.TemporaryDirectory()
        self.log_file = os.path.join(self.temp_dir.name, "followup.log")

    def tearDown(self) -> None:
        """Cleanup the temporary directory after tests."""
        self.temp_dir.cleanup()

    def test_fast_job_completes(self) -> None:
        """Fast job should complete successfully."""
        result = schedule_jobs(
            jobs=[("job1", fast_job)],
            timeout=1.0,
            follow_up=default_follow_up_logger,
            log_file=self.log_file,
        )
        self.assertEqual(result[0]["job_id"], "job1")
        self.assertEqual(result[0]["status"], "ok")
        self.assertEqual(result[0]["result"], 42)

    def test_slow_job_times_out(self) -> None:
        """Slow job should be terminated and logged on timeout."""
        result = schedule_jobs(
            jobs=[("job2", slow_job)],
            timeout=0.5,
            follow_up=default_follow_up_logger,
            log_file=self.log_file,
        )
        self.assertEqual(result[0]["status"], "timeout")
        self.assertTrue(Path(self.log_file).exists())

        with open(self.log_file, encoding="utf-8") as f:
            log = f.read()
            self.assertIn("job2", log)
            self.assertIn("timeout", log)

    def test_error_job_captures_exception(self) -> None:
        """Job that raises an exception should be recorded as error."""
        result = schedule_jobs(
            jobs=[("job3", error_job)],
            timeout=1.0,
            follow_up=default_follow_up_logger,
            log_file=self.log_file,
        )
        self.assertEqual(result[0]["status"], "error")
        self.assertIn("RuntimeError", result[0]["error"])

    def test_invalid_timeout_raises(self) -> None:
        """Invalid timeout should raise ValueError."""
        with self.assertRaises(ValueError):
            schedule_jobs(
                jobs=[("job5", fast_job)],
                timeout=0,
                follow_up=default_follow_up_logger,
                log_file=self.log_file,
            )

    def test_empty_job_list_returns_empty(self) -> None:
        """An empty job list should return an empty result list."""
        result = schedule_jobs(
            jobs=[],
            timeout=1.0,
            follow_up=default_follow_up_logger,
            log_file=self.log_file,
        )
        self.assertEqual(result, [])

    def test_invalid_job_format_raises(self) -> None:
        """Invalid job format should raise ValueError."""
        with self.assertRaises(ValueError):
            schedule_jobs(
                jobs=["bad job"],
                timeout=1.0,
                follow_up=default_follow_up_logger,
                log_file=self.log_file,
            )

    def test_follow_up_logger_writes_correct_line(self) -> None:
        """Follow-up logger should write a valid log entry."""
        default_follow_up_logger(
            job_id="job6",
            reason="timeout",
            duration_sec=2.3,
            log_file=self.log_file,
        )
        with open(self.log_file, encoding="utf-8") as f:
            line = f.read()
        self.assertIn("job6", line)
        self.assertIn("timeout", line)

    def test_multiple_jobs_mixed_status(self) -> None:
        """Multiple jobs should return mixed statuses correctly."""
        jobs = [
            ("job8", fast_job),
            ("job9", slow_job),
            ("job10", error_job),
        ]
        result = schedule_jobs(
            jobs=jobs,
            timeout=1.0,
            follow_up=default_follow_up_logger,
            log_file=self.log_file,
        )
        statuses = [r["status"] for r in result]
        self.assertEqual(statuses, ["ok", "timeout", "error"])

    def test_job_duration_accuracy(self) -> None:
        """Job duration should be a positive float."""
        result = schedule_jobs(
            jobs=[("job11", fast_job)],
            timeout=1.0,
            follow_up=default_follow_up_logger,
            log_file=self.log_file,
        )
        duration = result[0]["duration_sec"]
        self.assertIsInstance(duration, float)
        self.assertGreater(duration, 0.0)

    def test_duplicate_job_ids_allowed(self) -> None:
        """Duplicate job IDs are allowed and results preserved in order."""
        jobs = [("job12", fast_job), ("job12", slow_job)]
        result = schedule_jobs(
            jobs=jobs,
            timeout=0.5,
            follow_up=default_follow_up_logger,
            log_file=self.log_file,
        )
        self.assertEqual(len(result), 2)
        self.assertEqual(result[0]["status"], "ok")
        self.assertEqual(result[1]["status"], "timeout")

    def test_log_file_created_only_on_timeout(self) -> None:
        """Log file should not exist if no timeouts occurred."""
        schedule_jobs(
            jobs=[("job13", fast_job)],
            timeout=1.0,
            follow_up=default_follow_up_logger,
            log_file=self.log_file,
        )
        self.assertFalse(Path(self.log_file).exists())

    def test_custom_follow_up_called_on_timeout(self) -> None:
        """Custom follow-up should be invoked only on timeout."""

        self.called = False

        def mock_follow_up(job_id: str, reason: str,
                           duration_sec: float, log_file: str) -> None:
            self.called = True
            self.assertEqual(job_id, "job14")
            self.assertEqual(reason, "timeout")
            self.assertTrue(duration_sec >= 0.5)

        schedule_jobs(
            jobs=[("job14", slow_job)],
            timeout=0.5,
            follow_up=mock_follow_up,
            log_file=self.log_file,
        )
        self.assertTrue(self.called)


# Model Breaking Proof

#### Model Breaking Task URL: https://rlhf-v3.turing.com/prompt/20e1ecc0-ac56-484e-9e75-f847f770f3b5

#### Model code:

```python
import os
import time
import traceback  # <Issue> Unused import. Harmless, but should be removed for cleanliness.</Issue>
from datetime import datetime
from multiprocessing import Process, Queue
from collections.abc import Callable
from typing import Any


def run_job_with_timeout(func: Callable[[], Any], timeout: float) -> tuple[str, float, Any | BaseException]:
    """Run func in a child process with a hard timeout.
    Returns (status, duration_sec, payload) where:
      - status is "ok", "timeout", or "error"
      - payload is result (for "ok") or the exception instance (for "error")
    """
    def target_func(q):
        try:
            result = func()
            q.put(('ok', result))
        except Exception as e:
            q.put(('error', e))  # <Issue> If the first put fails due to non-picklable result, this second put can also fail; parent later blocks on queue.get(). Add timeout and a fallback error in parent, and consider a nested try/except here.</Issue>
    
    queue = Queue()
    process = Process(target=target_func, args=(queue,))
    start_time = time.monotonic()
    process.start()
    process.join(timeout)
    
    duration_sec = time.monotonic() - start_time
    
    if process.is_alive():
        process.terminate()
        process.join()
        return "timeout", duration_sec, None  # <Issue> Missing queue.close()/join_thread(). Also OK to return None payload for timeout.</Issue>
    
    status, payload = queue.get()  # <Issue> Blocking get() without timeout can deadlock if child failed to put anything.</Issue>
    return status, duration_sec, payload


def default_follow_up_logger(job_id: str, reason: str, duration_sec: float, log_file: str) -> None:
    """Append a line describing the timed-out job to log_file."""
    os.makedirs(os.path.dirname(log_file), exist_ok=True)  # <Issue> If log_file has no directory (e.g., "log.txt"), dirname is "", and makedirs("") raises FileNotFoundError. Should be conditional or use pathlib.</Issue>
    with open(log_file, 'a', encoding='utf-8') as file:
        timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        file.write(f"{timestamp} - Job ID: {job_id}, Reason: {reason}, Elapsed Time: {duration_sec:.2f} seconds\n")


def schedule_jobs(
    jobs: list[tuple[str, Callable[[], Any]]],
    timeout: float,
    follow_up: Callable[[str, str, float, str], None],
    log_file: str,
) -> list[dict]:
    """Run jobs sequentially with timeout and follow-up on timeout.
    Returns a list of dicts with keys: job_id, status, duration_sec, result or error.
    """
    if not isinstance(jobs, list) or not all(isinstance(job, tuple) and len(job) == 2 and isinstance(job[0], str) and callable(job[1]) for job in jobs):
        raise ValueError("invalid jobs")
    
    if not isinstance(timeout, float) or timeout <= 0:
        raise ValueError("invalid timeout")  # <Issue> Strict float check matches spec; optionally accept ints and cast for usability, but not required.</Issue>
    
    if not (callable(follow_up) and isinstance(log_file, str) and log_file):
        raise ValueError("invalid follow_up or log_file")
    
    records = []
    
    for job_id, func in jobs:
        status, duration, result = run_job_with_timeout(func, timeout)
        
        record = {'job_id': job_id, 'status': status, 'duration_sec': duration}
        
        if status == "ok":
            record['result'] = result
        elif status == "error":
            record['error'] = repr(result)
        elif status == "timeout":
            follow_up(job_id, "timeout", duration, log_file)  # <Issue> OK; but if follow-up raises due to path bug above, exception should propagate (per spec). Fix the logger.</Issue>
        
        records.append(record)
    
    return records

# Example job functions and usage

def slow_job():
    import time
    time.sleep(2)
    return "done"

def fast_job():
    return 42

if __name__ == "__main__":
    records = schedule_jobs(
        jobs=[("J1", fast_job), ("J2", slow_job)],
        timeout=1.0,
        follow_up=default_follow_up_logger,
        log_file="job_followups.log",
    )
    print(records)  # <Issue> Outside the scheduler, allowed by spec. Keep guarded by __main__.</Issue>
```